In [27]:
import numpy as np

from scipy import interpolate

from utils import check_loc_ast, in_box, get_orbits, calc_obs_ctime, make_box

from pixell import utils, bunch, enmap

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
ifile = '/home/r/rbond/sigurdkn/project/actpol/maps/depth1/release/15385/depth1_1538528823_pa4_f150_map.fits'

astinfo = np.load('/gpfs/fs0/project/r/rbond/sigurdkn/actpol/ephemerides/objects/Vesta.npy').view(np.recarray)

In [3]:
orbit = interpolate.interp1d(astinfo.ctime, [utils.unwind(astinfo.ra*utils.degree), astinfo.dec*utils.degree], kind=3)

In [4]:
infofile = utils.replace(ifile, "map.fits", "info.hdf")
tfile    = utils.replace(ifile, "map.fits", "time.fits")
varfile    = utils.replace(ifile, "map.fits", "ivar.fits")
rhofile    = utils.replace(ifile, "map.fits", "rho.fits")
kfile    = utils.replace(ifile, "map.fits", "kappa.fits")

In [30]:
info = bunch.read(infofile)

# Get the asteroid coordinates
ctime0   = np.mean(info.period)
adata0   = orbit(ctime0)
ast_pos0 = utils.rewind(adata0[1::-1])
message  = "%.0f  %8.3f %8.3f  %8.3f %8.3f %8.3f %8.3f" % (info.t, ast_pos0[1]/utils.degree, ast_pos0[0]/utils.degree, info.box[0,1]/utils.degree, info.box[1,1]/utils.degree, info.box[0,0]/utils.degree, info.box[1,0]/utils.degree)

full_box  = make_box(ast_pos0, 10*utils.arcmin)
# Read it and check if we have enough hits in the area we want to use
try:
        tmap = enmap.read_map(tfile, box=full_box)
        tmap[tmap!=0] += info.t
except (TypeError, FileNotFoundError):
        print("Error reading %s. Skipping" % ifile)
# Break out early if nothing is hit

# Figure out what time the asteroid was actually observed
ctime, err = calc_obs_ctime(orbit, tmap, ctime0)

# Now that we have the proper time, get the asteroids actual position
adata    = orbit(ctime)
ast_pos  = utils.rewind(adata[1::-1])                

In [31]:
ast_pos

array([[-0.44839922],
       [-1.48186142]])

In [37]:
(ast_pos-np.reshape(ast_pos0, (2,1)))/utils.arcmin

array([[ 0.09880967],
       [-4.28265618]])

In [6]:
in_box(info.box, ast_pos0)

True

In [39]:
ctime

array([1.53853005e+09])

In [80]:
check_loc_ast(np.array([ast_pos[0],ast_pos[0]]), np.array([ast_pos[1],ast_pos[1]]), np.ones(2)*ctime,
              '/gpfs/fs0/project/r/rbond/sigurdkn/actpol/ephemerides/objects/', max_idx=5)

True

In [46]:
test = get_orbits('/gpfs/fs0/project/r/rbond/sigurdkn/actpol/ephemerides/objects/', max_idx=5)

In [48]:
utils.rewind(test[3](ctime)[1::-1])

array([[-0.44839922],
       [-1.48186142]])

In [62]:
ast_locs = np.zeros((1,len(test), 2)) #an individual interp returns [ra, dec] so last dim is 2 
for i in range(1):
    for j, orbit in enumerate(test):
        ast_locs[i][j] = np.reshape(utils.rewind(orbit(ctime)[1::-1]), ast_locs[i][j].shape)

print(ast_locs)



[[[ 0.11874221  1.69420721]
  [-0.05004571  2.82333888]
  [ 0.03207277 -2.90303034]
  [-0.44839922 -1.48186142]
  [ 0.08230682  1.06739043]
  [ 0.03478177  2.88482787]]]


In [45]:
utils.rewind(orbit(ctime)[1::-1])

array([[-0.44839922],
       [-1.48186142]])

In [68]:
ast_pos - np.reshape(ast_locs[0][3], ast_pos.shape)

array([[0.],
       [0.]])

In [67]:
ast_pos.shape

(2, 1)